In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx
from networkx import edge_betweenness_centrality as betweenness
from networkx.algorithms.community.centrality import girvan_newman
from modularity_maximization import partition

from modularity_maximization.utils import get_modularity
from networkx.algorithms.community import greedy_modularity_communities
from cdlib import algorithms
import pickle

# import data
df = pd.read_csv('../dataset/edge_mean_th95.csv', index_col =0, dtype='int64')

/usr/local/lib/python3.6/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
dataset = '../dataset/edge_mean_th95.csv'

# import data
df = pd.read_csv(dataset, index_col =0, dtype='int64')

# define edge_list (u,v)
edge_list = list(df.itertuples(index=False, name=None))

# define node_list
node_list = [x for x in range(max(df['dst']))]

# Create Graph 
G = nx.Graph()

# Add node and edge to graph
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

# Delete node which does not have neighborhood
print('Delete node with no neighborhood: {} \n'.format(list(nx.isolates(G))))
G.remove_nodes_from(list(nx.isolates(G)))
print('Number of node:',len(G.nodes))

Delete node with no neighborhood: [12, 14, 16, 18, 21, 24, 25, 27, 31, 44, 47, 49, 51, 52, 53, 57, 60, 61, 63, 64, 65, 71, 72, 73, 74, 77, 79, 80, 81, 84, 87, 88, 92, 93, 99, 101, 103, 104, 105, 107, 109, 112, 113, 116, 117, 124, 128, 131, 133, 138, 140] 

Number of node: 91


In [4]:
# criterion for most valuable edge (can be changed)
def most_central_edge(G):
    centrality = betweenness(G, weight="weight")
    # print(centrality)
    return max(centrality, key=centrality.get)

# Eigenvector 
Partition 자체가 Modularity Matrix에 대한 Eigenvector

In [5]:
nx.info(G)

'Name: \nType: Graph\nNumber of nodes: 91\nNumber of edges: 169\nAverage degree:   3.7143'

In [6]:
comm_dict = partition(G)

for comm in set(comm_dict.values()):
    print("Community %d"%comm)
    print(', '.join([str(node) for node in comm_dict if comm_dict[node] == comm]))
    
print('Modularity of such partition for karate is %.3f' % get_modularity(G, comm_dict))

KeyboardInterrupt: 

# Modularity Maximization with Greedy Algorithm

In [ ]:
nx.info(G)

In [ ]:
# greedy_modularity_communities(G)

In [ ]:
greedy_module_comms = nx.algorithms.community.greedy_modularity_communities(G)

In [ ]:
greedy_dict = {}

for i, value in enumerate(greedy_module_comms):
    values = list(value)
    for v in values:
        greedy_dict[v] = i

In [ ]:
for comm in set(greedy_dict.values()):
    print("Community %d"%comm)
    print(comm)
    print(', '.join([str(node) for node in greedy_dict if greedy_dict[node] == comm]))

In [ ]:
print('Modularity of such partition for karate is %.3f' % get_modularity(G, greedy_dict))

In [ ]:
plt.figure(figsize=(10,10))
pos = nx.spring_layout(G, scale=4)
nx.draw(G, pos, node_color=range(len(G.nodes)), node_size=1000, with_labels=True, cmap=plt.cm.Blues)
plt.title('Community Detection with betweenness')
plt.show()
plt.savefig('betweenness.jpg')

# Performance Metric

* Betweenness > Girvan-Newman Algorithm

* Partition > Newman 
https://github.com/zhiyzuo/python-modularity-maximization/blob/master/modularity_maximization/community_newman.py
https://arxiv.org/abs/physics/0602124

* modularity_max greedy_modularity communities
Clauset-Newman-Moore greedy modularity maximization

In [7]:
edge_mean_th95 = '../dataset_v2/edge_mean_th95_v2.csv' 
edge_mean_top5 = '../dataset_v2/edge_mean_top5_v2.csv'
edge_mice_th95 = '../dataset_v2/edge_mice_th95_v2.csv'
edge_mice_top5 = '../dataset_v2/edge_mice_top5_v2.csv'
edge_grape_th95 = '../dataset_v2/edge_GRAPE_th95_v2.csv'
edge_grape_top5 = '../dataset_v2/edge_GRAPE_top5_v2.csv'
edge_original_th95 = '../dataset/edge_Original_th95.csv'
edge_original_top5 = '../dataset/edge_Original_top5.csv'

In [8]:
datalist = [edge_original_th95, edge_original_top5, edge_mean_th95, edge_mean_top5, edge_mice_th95, edge_mice_top5, edge_grape_th95, edge_grape_top5]

In [9]:
# criterion for most valuable edge (can be changed)
def most_central_edge(G):
    centrality = betweenness(G, weight="weight")
    # print(centrality)
    return max(centrality, key=centrality.get)

In [10]:
def modularity_calculation(dataset):
    print('='*50)
    dataset_name = dataset.split("/")[-1]
    print(f'[[Dataset: {dataset_name}]]')
    # import data
    df = pd.read_csv(dataset, index_col =0, dtype='int64')
    df = df.rename(columns={'0': 'dst', '1':'src'})

    # define edge_list (u,v)
    edge_list = list(df.itertuples(index=False, name=None))

    # define node_list
    node_list = [x for x in range(max(df['dst']))]

    # Create Graph 
    G = nx.Graph()

    # Add node and edge to graph
    G.add_nodes_from(node_list)
    G.add_edges_from(edge_list)
    
#     Delete node which does not have neighborhood
#     print('Delete node with no neighborhood: {} \n'.format(list(nx.isolates(G))))
#     G.remove_nodes_from(list(nx.isolates(G)))
#     print('Number of node:',len(G.nodes))
    
    print('{Partition > Eigenvector based clustering}')
    eigen_dict = partition(G)

#     for comm in set(comm_dict.values()):
#         print("Community %d"%comm)
#         print(', '.join([str(node) for node in comm_dict if comm_dict[node] == comm]))
        
    print(f': Modularity of such partition for {dataset.split("/")[-1]} is \033[1m %.3f \033[0m' % get_modularity(G, eigen_dict))
    print()
    print('-'*20)
    print()
    print('{Modularity Maximization}')
    
    greedy_module_comms = nx.algorithms.community.greedy_modularity_communities(G)
    greedy_dict = {}

    for i, value in enumerate(greedy_module_comms):
        values = list(value)
        for v in values:
            greedy_dict[v] = i
    
    print(f': Modularity of such partition for {dataset.split("/")[-1]} is \033[1m %.3f \033[0m' % get_modularity(G, greedy_dict))
    print()
    print('-'*20)
    print()
    print('{Betweenness}')
    
    communities = girvan_newman(G, most_valuable_edge=most_central_edge)

    node_groups = []
    for com in next(communities):
        node_groups.append(list(com))

    between_dict = {}
    for i, value in enumerate(node_groups):
        for v in value:
            between_dict[v] = i
    
    print(f': Modularity of such partition for {dataset.split("/")[-1]} is \033[1m %.3f \033[0m' % get_modularity(G, between_dict))
    print()
    print('-'*20)
    print()
    print('{Walk Trap}')
    
    walktrap = algorithms.walktrap(G)
    walk_comms=walktrap.communities
    walktrap_dict = {}

    for i, value in enumerate(walk_comms):
        values = list(value)
        for v in values:
            walktrap_dict[v] = i
            
    total_data = {'eigen': eigen_dict,
                  'greedy': greedy_dict,
                  'between': between_dict,
                  'walktrap': walktrap_dict} 
    
    print(f': Modularity of such partition for {dataset.split("/")[-1]} is \033[1m %.3f \033[0m' % get_modularity(G, walktrap_dict))
    with open(f'{dataset_name}.pickle', 'wb') as f:
        pickle.dump(total_data, f)

In [11]:
for data in datalist:
    modularity_calculation(data)

[[Dataset: edge_Original_th95.csv]]
{Partition > Eigenvector based clustering}
Calculating modularity for undirected graph
: Modularity of such partition for edge_Original_th95.csv is  0.783 

--------------------

{Modularity Maximization}
Calculating modularity for undirected graph
: Modularity of such partition for edge_Original_th95.csv is  0.783 

--------------------

{Betweenness}
Calculating modularity for undirected graph
: Modularity of such partition for edge_Original_th95.csv is  0.720 

--------------------

{Walk Trap}
Calculating modularity for undirected graph
: Modularity of such partition for edge_Original_th95.csv is  0.760 
[[Dataset: edge_Original_top5.csv]]
{Partition > Eigenvector based clustering}
Calculating modularity for undirected graph
: Modularity of such partition for edge_Original_top5.csv is  0.757 

--------------------

{Modularity Maximization}
Calculating modularity for undirected graph
: Modularity of such partition for edge_Original_top5.csv is  0

InternalError: Error at src/community/community_misc.c:116: Number of steps is greater than number of rows in merges matrix: found 141 steps, 139 rows. -- Invalid value